In [1]:
import requests

# 1. The Endpoint URL
url = "https://data.renfe.com/api/3/action/datastore_search"

# 2. The Parameters (The Resource ID is required)
params = {
    "resource_id": "a2368cff-1562-4dde-8466-9635ea3a572a",
    "limit": 5,       # Limit to 5 results
    "q": "jones"      # Optional: Search for this word
}

# 3. Make the Request
response = requests.get(url, params=params)

# 4. Check and Parse the Data
if response.status_code == 200:
    data = response.json()
    
    # Drill down to the actual records
    records = data['result']['records']
    
    print(f"Found {len(records)} records:")
    for record in records:
        print(record)
else:
    print("Something went wrong.")

Found 0 records:


In [4]:
import requests
import json

# 1. The URL
url = "https://data.renfe.com/api/3/action/datastore_search"

# 2. The Parameters
# I REMOVED the line 'q': 'jones'
params = {
    "resource_id": "a2368cff-1562-4dde-8466-9635ea3a572a",
    "limit": 10
}

# 3. The Request
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    records = data['result']['records']
    
    print(f"Success! Found {len(records)} records.\n")

    for item in records:
        name = item.get("DESCRIPCION")
        # Unpacking the "Fichas" pocket
        pdf_link = item.get("Fichas") 
        
        print(f"Station: {name}")
        print(f"  -> More info: {pdf_link}")
        print("-" * 30) # A separator line

else:
    print(f"Error: {response.status_code}")

Success! Found 10 records.

Station: AEROPUERTO
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54505.pdf
------------------------------
Station: ALJAIMA
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54407.pdf
------------------------------
Station: ALORA
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54405.pdf
------------------------------
Station: BENALMADENA-ARROYO DE LA MIEL
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54511.pdf
------------------------------
Station: CAMPANILLAS
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54410.pdf
------------------------------
Station: CARTAMA
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54408.pdf
------------------------------
Station: CARVAJAL
  -> More info: https://data.renfe.com/files/estaciones/FichaEstacion54513.pdf
------------------------------
Station: EL PINILLO
  -> More info: https://data.renfe.

# Static GTFS

This is the ZIP file. You update this Weekly (or nightly). This gives you the base schedule (Routes, Stops, Planned Times).

In [3]:
import requests
import os

url = "https://data.renfe.com/api/3/action/resource_show?id=25d6b043-9e47-4f99-bd91-edd51d782450"

# 1. Ask the API for the metadata
response = requests.get(url)
data = response.json()

# 2. Get the "Last Modified" date from Renfe
server_last_modified = data['result']['last_modified']
print(f"Server version is from: {server_last_modified}")

# 3. Compare with your local file (if you have one)
local_file_path = "renfe_schedule.zip"

if os.path.exists(local_file_path):
    # Get the time you last downloaded your local file
    # (This is a simple way; in a real app, you'd save the string above to a text file)
    local_timestamp = os.path.getmtime(local_file_path)
    from datetime import datetime
    print(f"You last downloaded on: {datetime.fromtimestamp(local_timestamp)}")
    
    # LOGIC: 
    # If server_date is newer than local_date -> DOWNLOAD
    # Else -> DO NOTHING
else:
    print("You don't have the file yet. You should download it.")

Server version is from: 2025-11-15T00:40:21
You last downloaded on: 2025-11-15 18:25:24.781563


In [8]:
import requests

# 1. The Endpoint (Notice we changed the action!)
# FROM: datastore_search
# TO:   resource_show
url = "https://data.renfe.com/api/3/action/resource_show"

# 2. The Parameters
# For this action, we just need the ID.
params = {
    "id": "25d6b043-9e47-4f99-bd91-edd51d782450"
}

# 3. Make the Request
response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()
    
    # 4. Extract the Download URL
    # The API gives us the "metadata" (info about the file).
    # We want the specific key called "url".
    file_url = data['result']['url']
    file_format = data['result']['format']
    
    print(f"Found the file!")
    print(f"Format: {file_format}")
    print(f"Download Link: {file_url}")
    
    # OPTIONAL: If you want Python to actually download and save the zip file:
    doc = requests.get(file_url)
    with open("renfe_schedule.zip", 'wb') as f:
        f.write(doc.content)
    print("File downloaded to your computer.")

else:
    print("Error finding resource.")

Found the file!
Format: GTFS
Download Link: https://ssl.renfe.com/gtransit/Fichero_AV_LD/google_transit.zip
File downloaded to your computer.


In [1]:
import pandas as pd
import os

# Point this to your folder path shown in the screenshot
# e.g., "C:/Users/tomso/workspace/renfe_mcp/renfe_schedule"
folder_path = "C:/Users/tomso/workspace/renfe_mcp/renfe_schedule" 

# Let's load the STOPS file
stops_df = pd.read_csv(os.path.join(folder_path, "stops.txt"))

# Show the first 5 rows
print(stops_df.head())

   stop_id  stop_code                         stop_name  stop_desc   stop_lat  \
0    17000        NaN  Madrid-Chamartín-Clara Campoamor        NaN  40.471179   
1    37606        NaN                           Badajoz        NaN  38.890696   
2    37603        NaN                           Montijo        NaN  38.913539   
3    37500        NaN                            Mérida        NaN  38.921503   
4    35400        NaN                           Cáceres        NaN  39.461131   

   stop_lon  zone_id  stop_url  location_type  parent_station  stop_timezone  \
0 -3.682952      NaN       NaN            NaN             NaN  Europe/Madrid   
1 -6.981753      NaN       NaN            NaN             NaN  Europe/Madrid   
2 -6.598577      NaN       NaN            NaN             NaN  Europe/Madrid   
3 -6.343794      NaN       NaN            NaN             NaN  Europe/Madrid   
4 -6.385679      NaN       NaN            NaN             NaN  Europe/Madrid   

   wheelchair_boarding          

In [2]:
# 2. Load the Routes file
routes_df = pd.read_csv(os.path.join(folder_path, "routes.txt"))

# 3. Take a peek
# We use .head(10) to see the first 10 rows
print(routes_df.head(10))

# Optional: Let's see what 'Types' of trains exist
# This prints the unique names of the lines found
print("\n--- List of Service Names ---")
print(routes_df['route_long_name'].unique()[:10]) # Printing just the first 10 unique names

          route_id  agency_id route_short_name  route_long_name  route_desc  \
0  1700037606GL023       1071            ALVIA              NaN         NaN   
1  1700056312GL026       1071        Intercity              NaN         NaN   
2  1700050300GL026       1071        Intercity              NaN         NaN   
3  5631217000GL026       1071        Intercity              NaN         NaN   
4  5030017000GL026       1071        Intercity              NaN         NaN   
5  1120013200GL026       1071        Intercity              NaN         NaN   
6  1151111208GL026       1071        Intercity              NaN         NaN   
7  1120811511GL026       1071        Intercity              NaN         NaN   
8  1320011200GL026       1071        Intercity              NaN         NaN   
9  3760617000GL023       1071            ALVIA              NaN         NaN   

   route_type  route_url route_color  \
0           2        NaN      F2F5F5   
1           2        NaN      F2F5F5   
2         

# Real-Time GTFS

This is the other API you found (trip_updates.json). You update this Every 30 Seconds.

In [7]:
import requests
import time # We import time to simulate the "Live" aspect

# 1. Get the Link (Just like the ZIP file approach)
url = "https://data.renfe.com/api/3/action/resource_show"
params = {
    "id": "d772c2a8-26e8-4419-b25a-5967fe279c8f" # You need to copy the ID from the URL bar or metadata
}

response = requests.get(url, params=params)
data = response.json()
real_time_url = data['result']['url']

print(f"Connected to the Live Feed: {real_time_url}")

# 2. Fetch the Live Data
# Since it's JSON, we can read it directly into Python
live_response = requests.get(real_time_url)
live_data = live_response.json()

# 3. What's inside?
# Usually these files (GTFS-Realtime) have a list of "entities" (trains)
number_of_trains = len(live_data.get('entity', []))

print(f"Right now, there are {number_of_trains} active updates in the system.")

# If you were building an app, you would put step #2 inside a loop 
# that runs every 30 seconds to update the map!

Connected to the Live Feed: https://gtfsrt.renfe.com/trip_updates_LD.json
Right now, there are 54 active updates in the system.
